In [1]:
from transformers import pipeline
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report


In [2]:
sentiment_model = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [5]:
# Upload the data
from google.colab import files
uploaded = files.upload()

Saving labeled_2016_2024_test_data.csv to labeled_2016_2024_test_data.csv


In [6]:
df = pd.read_csv("2016_2024_NYT_articles.csv")
df_test = pd.read_csv("labeled_2016_2024_test_data.csv")

df.drop(columns=["Unnamed: 0"], inplace=True)
df.set_index("date", inplace=True)
df.head()


,text,label
date,,
2016-01-04,"General Motors, Gazing at Future, Invests $500...",NaN
2016-01-04,G.M.’s Investment in Lyft Hedges Against a Fut...,NaN
2016-01-05,Carmakers Set to Grab Spotlight at Internation...,NaN
2016-01-06,"On Display at CES, Tech Ideas in Their Awkward...",NaN
2016-01-07,Carmakers Forge Ahead With Electric Vehicles ...,NaN


**Test model on labeled data**

In [7]:
df_test['label'] = df_test['label'].replace(0, -1)


sentiment_test = sentiment_model(df_test['text'].tolist())

# Extract sentiment labels (positive/negative) and confidence scores
df_test['predicted_label'] = [result['label'] for result in sentiment_test]
df_test['predicted_score'] = [result['score'] for result in sentiment_test]

# Map sentiment labels to numerical values (1 for positive, -1 for negative)
df_test['predicted_sentiment'] = df_test['predicted_label'].map({'POSITIVE': 1, 'NEGATIVE': -1})

# Verify the updated dataset
print(df_test)

          date                                               text  label  \
0   2017-06-07  Why Car Companies Are Hiring Computer Security...     -1   
1   2016-11-03  Whose Life Should Your Car Save? It can be pro...     -1   
2   2016-12-07  The Robot Revolution Will Be the Quietest One ...     -1   
3   2016-12-17  Uber Defies California Regulators With Self-Dr...     -1   
4   2018-03-20  How a Self-Driving Uber Killed a Pedestrian in...     -1   
5   2018-03-19  Self-Driving Uber Car Kills Pedestrian in Ariz...     -1   
6   2018-02-27  Automated Vehicles Can’t Save Cities They coul...     -1   
7   2017-03-25  Uber Suspends Tests of Self-Driving Vehicles A...     -1   
8   2017-05-28  The Future of European Transit: Driverless and...     -1   
9   2018-12-07  Taxpayers Always Lose Industry’s Shell Game Wi...     -1   
10  2017-11-08  The Rev-Up: Imagining a 20% Self-Driving World...      1   
11  2017-09-28  In Taiwan, Modest Test of Driverless Bus May H...      1   
12  2016-09-

In [8]:
accuracy = accuracy_score(df_test['label'], df_test['predicted_sentiment'])
print(f'Accuracy: {accuracy}')

# Generate a classification report
report = classification_report(df_test['label'], df_test['predicted_sentiment'])
print(f'Classification Report:\n{report}')

Accuracy: 0.8
Classification Report:
              precision    recall  f1-score   support

          -1       0.80      0.80      0.80        10
           1       0.80      0.80      0.80        10

    accuracy                           0.80        20
   macro avg       0.80      0.80      0.80        20
weighted avg       0.80      0.80      0.80        20



In [9]:
# Predict sentiment for each headline
sentiment_results = sentiment_model(df['text'].tolist())

# Extract sentiment labels (positive/negative) and confidence scores
df['sentiment_label'] = [result['label'] for result in sentiment_results]
df['sentiment_score'] = [result['score'] for result in sentiment_results]

# Map sentiment labels to numerical values (1 for positive, -1 for negative)
df['sentiment'] = df['sentiment_label'].map({'POSITIVE': 1, 'NEGATIVE': -1})

# Verify the updated dataset
print(df.head())

                                                         text label  \
date                                                                  
2016-01-04  General Motors, Gazing at Future, Invests $500...   NaN   
2016-01-04  G.M.’s Investment in Lyft Hedges Against a Fut...   NaN   
2016-01-05  Carmakers Set to Grab Spotlight at Internation...   NaN   
2016-01-06  On Display at CES, Tech Ideas in Their Awkward...   NaN   
2016-01-07  Carmakers Forge Ahead With Electric Vehicles  ...   NaN   

           sentiment_label  sentiment_score  sentiment  
date                                                    
2016-01-04        POSITIVE         0.936707          1  
2016-01-04        POSITIVE         0.843959          1  
2016-01-05        POSITIVE         0.997886          1  
2016-01-06        NEGATIVE         0.998428         -1  
2016-01-07        POSITIVE         0.996091          1  


Save labeled data

In [10]:
df.reset_index(inplace=True)
print(df.head())


         date                                               text label  \
0  2016-01-04  General Motors, Gazing at Future, Invests $500...   NaN   
1  2016-01-04  G.M.’s Investment in Lyft Hedges Against a Fut...   NaN   
2  2016-01-05  Carmakers Set to Grab Spotlight at Internation...   NaN   
3  2016-01-06  On Display at CES, Tech Ideas in Their Awkward...   NaN   
4  2016-01-07  Carmakers Forge Ahead With Electric Vehicles  ...   NaN   

  sentiment_label  sentiment_score  sentiment  
0        POSITIVE         0.936707          1  
1        POSITIVE         0.843959          1  
2        POSITIVE         0.997886          1  
3        NEGATIVE         0.998428         -1  
4        POSITIVE         0.996091          1  


In [11]:
df.to_csv('2016_predicted_sentiment.csv', index=False)


In [12]:
# Download the CSV file
files.download('2016_predicted_sentiment.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [38]:
df.head()

,date,text,label,sentiment_label,sentiment_score,sentiment
0,2016-01-04,"General Motors, Gazing at Future, Invests $500...",NaN,POSITIVE,0.936707,1
1,2016-01-04,G.M.’s Investment in Lyft Hedges Against a Fut...,NaN,POSITIVE,0.843959,1
2,2016-01-05,Carmakers Set to Grab Spotlight at Internation...,NaN,POSITIVE,0.997886,1
3,2016-01-06,"On Display at CES, Tech Ideas in Their Awkward...",NaN,NEGATIVE,0.998428,-1
4,2016-01-07,Carmakers Forge Ahead With Electric Vehicles ...,NaN,POSITIVE,0.996091,1
